In [1]:
# NF‑κB Luciferase Reporter Assay – pylabrobot version
# Author: Guangxin Zhang 
# This script automates the luciferase activity measurement protocol using
# pylabrobot and the opentrons backend .

from pylabrobot.resources.opentrons import OTDeck
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends.opentrons_backend_jump import OpentronsBackend
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.resources import Coordinate



from pylabrobot.resources.opentrons import (
corning_96_wellplate_360ul_flat,
    nest_12_reservoir_15ml,
    nest_1_reservoir_195ml,
    opentrons_96_tiprack_300ul
)

# ──────────────────────────────────────
# User‑configurable constants (µL)
MEDIUM_VOL = 100     # volume of spent medium to remove
PBS_VOL    = 50      # PBS wash volume
LYSIS_VOL  = 30      # lysis buffer volume
LUC_VOL    = 100     # luciferase reagent volume

# ──────────────────────────────────────
def _build_deck(lh: LiquidHandler):
    """Load all labware on the deck and return handy shortcuts."""
    # Tip‑racks on slots 8, 11, 1, 4  (order preserved to match Opentrons layout)
    tiprack_slots = [8, 11, 1, 4]
    tipracks = []
    # Load tip racks
    for slot_i in tiprack_slots:
      tr = opentrons_96_tiprack_300ul(name=f"tiprack_{slot_i}")
      lh.deck.assign_child_at_slot(tr, slot=slot_i)
      tipracks.append(tr)
    # Working 96‑well plate at slot 6
    working_plate = corning_96_wellplate_360ul_flat(name="working_plate")
    lh.deck.assign_child_at_slot(working_plate, slot=6)

    # 12‑channel reservoir (PBS, Lysis, Luciferase) at slot 3
    reagent_stock = nest_12_reservoir_15ml(name="reagent_stock")
    lh.deck.assign_child_at_slot(reagent_stock, slot=3)

    # 1‑channel waste reservoir at slot 9
    waste_res = nest_1_reservoir_195ml(name="waste_res")
    lh.deck.assign_child_at_slot(waste_res, slot=9)

    return {
        "tip_racks"     : tipracks,
        "working_plate" : working_plate,
        "reagent_res"   : reagent_stock,
        "waste_res"     : waste_res
    }

def _tip_gen(tip_racks):
    """Yield the next available tip."""
    for rack in tip_racks:
        for tip in rack:
            yield tip
    raise RuntimeError("Out of tips!")


In [2]:
backend = OpentronsBackend(host="localhost", simulation=True)
lh = LiquidHandler(backend=backend, deck=OTDeck())
deck = _build_deck(lh)
await lh.setup()
vis = Visualizer(resource=lh)
await vis.setup()
tips = _tip_gen(deck["tip_racks"])

Websocket server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


In [3]:
working_plate_volumns = [('culture medium', MEDIUM_VOL)] * 12 + [(None, 0)] * (96-12)
deck["working_plate"].set_well_liquids(working_plate_volumns)

reagent_info = [('PBS Buffer', 5000), ('Lysis Buffer', 5000), ('Luciferase Reagent', 5000)]+[ (None, 0) ]* 9
deck["reagent_res"].set_well_liquids(reagent_info)

pbs        = deck["reagent_res"][0][0]
lysis      = deck["reagent_res"][1][0]
luciferase = deck["reagent_res"][2][0]
waste_res  = deck["waste_res"][0]

wells_name = [f"A{i}" for i in range(1, 13)]
cells_all  = deck["working_plate"][wells_name] # A1–A12


In [4]:
from pylabrobot.resources import set_tip_tracking, set_volume_tracking
set_tip_tracking(True), set_volume_tracking(True)

(None, None)

In [5]:
# ────────── 1. Remove spent medium ──────────
for cell in cells_all:
    tip = next(tips)
    await lh.pick_up_tips(tip)
    await lh.aspirate([cell], [MEDIUM_VOL * 1.2], flow_rates=[0.2])
    await lh.dispense(waste_res, [MEDIUM_VOL * 1.4], flow_rates=[3])
    await lh.discard_tips()

Tracker only has 100uL, please pay attention.
Container has too little liquid: 120.0uL > 100uL, please pay attention.
Air bubble detected, please pay attention.
[Well(name=waste_res_A1, location=Coordinate(010.480, 007.140, 004.550), size_x=106.8, size_y=71.2, size_z=25, category=well)]
Tracker only has 100uL, please pay attention.
Container has too little liquid: 140.0uL > 100uL, please pay attention.
Air bubble detected, please pay attention.
Tracker only has 100uL, please pay attention.
Container has too little liquid: 120.0uL > 100uL, please pay attention.
Air bubble detected, please pay attention.
[Well(name=waste_res_A1, location=Coordinate(010.480, 007.140, 004.550), size_x=106.8, size_y=71.2, size_z=25, category=well)]
Tracker only has 100uL, please pay attention.
Container has too little liquid: 140.0uL > 100uL, please pay attention.
Air bubble detected, please pay attention.
Tracker only has 0uL, please pay attention.
Container has too little liquid: 120.0uL > 0uL, please pay

In [6]:
# ────────── 2. PBS wash (add) ──────────
await lh.pick_up_tips(next(tips))
for cell in cells_all:
 #这里还没完全写好，后端需要改
  await lh.aspirate([pbs], vols=[PBS_VOL], flow_rate = [3], blow_out_air_volume = [20])
  await lh.dispense([cell], [PBS_VOL + 20], flow_rate=0.3,
                offset=Coordinate(z=-2))
await lh.discard_tips()


[Well(name=working_plate_A1, location=Coordinate(011.954, 071.814, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well)]
Tracker only has 50.0uL, please pay attention.
Container has too little liquid: 70.0uL > 50.0uL, please pay attention.
Air bubble detected, please pay attention.
[Well(name=working_plate_A2, location=Coordinate(020.954, 071.814, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well)]
Tracker only has 50.0uL, please pay attention.
Container has too little liquid: 70.0uL > 50.0uL, please pay attention.
Air bubble detected, please pay attention.
[Well(name=working_plate_A3, location=Coordinate(029.954, 071.814, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well)]
Tracker only has 50.0uL, please pay attention.
Container has too little liquid: 70.0uL > 50.0uL, please pay attention.
Air bubble detected, please pay attention.
[Well(name=working_plate_A4, location=Coordinate(038.955, 071.814, 003.550), size_x=4.851, size_y=4.851, size

/Users/guangxinzhang/Documents/Deep Potential/pylabrobot/pylabrobot/liquid_handling/liquid_handler.py:338: UserWarning: Extra arguments to backend.dispense: {'offset', 'flow_rate'}
  warnings.warn(f"Extra arguments to backend.{method.__name__}: {extra}")


In [7]:
 # ────────── 3. PBS wash (remove) ──────────
for cell in cells_all:
    tip = next(tips)
    await lh.pick_up_tips(tip)
    await lh.aspirate([cell], [PBS_VOL * 1.5], flow_rates=[0.2])
    await lh.dispense(waste_res, [PBS_VOL * 1.5], flow_rates=[3])
    await lh.discard_tips()

Tracker only has 50.0uL, please pay attention.
Container has too little liquid: 75.0uL > 50.0uL, please pay attention.
Air bubble detected, please pay attention.
[Well(name=waste_res_A1, location=Coordinate(010.480, 007.140, 004.550), size_x=106.8, size_y=71.2, size_z=25, category=well)]
Tracker only has 50.0uL, please pay attention.
Container has too little liquid: 75.0uL > 50.0uL, please pay attention.
Air bubble detected, please pay attention.
Tracker only has 50.0uL, please pay attention.
Container has too little liquid: 75.0uL > 50.0uL, please pay attention.
Air bubble detected, please pay attention.
[Well(name=waste_res_A1, location=Coordinate(010.480, 007.140, 004.550), size_x=106.8, size_y=71.2, size_z=25, category=well)]
Tracker only has 50.0uL, please pay attention.
Container has too little liquid: 75.0uL > 50.0uL, please pay attention.
Air bubble detected, please pay attention.
Tracker only has 50.0uL, please pay attention.
Container has too little liquid: 75.0uL > 50.0uL, p

In [8]:
# ────────── 4. Add lysis buffer ──────────
tip = next(tips)
await lh.pick_up_tips(tip)

for cell in cells_all:
    await lh.aspirate([lysis], [LYSIS_VOL], flow_rates=[0.5])
    #backend.delay(seconds=2)
    await lh.dispense([cell], [LYSIS_VOL], flow_rates=[0.3],
                offset=Coordinate(z=5))
    #backend.delay(seconds=2)
await lh.discard_tips()
# backend.delay(minutes=3)

[Well(name=working_plate_A1, location=Coordinate(011.954, 071.814, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well)]
[Well(name=working_plate_A2, location=Coordinate(020.954, 071.814, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well)]
[Well(name=working_plate_A3, location=Coordinate(029.954, 071.814, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well)]
[Well(name=working_plate_A4, location=Coordinate(038.955, 071.814, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well)]
[Well(name=working_plate_A5, location=Coordinate(047.955, 071.814, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well)]
[Well(name=working_plate_A6, location=Coordinate(056.955, 071.814, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well)]
[Well(name=working_plate_A7, location=Coordinate(065.954, 071.814, 003.550), size_x=4.851, size_y=4.851, size_z=10.67, category=well)]
[Well(name=working_plate_A8, location=Coordinate(074.95

/Users/guangxinzhang/Documents/Deep Potential/pylabrobot/pylabrobot/liquid_handling/liquid_handler.py:338: UserWarning: Extra arguments to backend.dispense: {'offset'}
  warnings.warn(f"Extra arguments to backend.{method.__name__}: {extra}")


In [9]:
# ────────── 5. Add luciferase reagent ──────────
for cell in cells_all:
    lh.pick_up_tips(next(tips))
    lh.aspirate([luciferase], [LUC_VOL], flow_rates=[0.75])
   # backend.delay(seconds=2)
    lh.dispense([cell], [LUC_VOL], flow_rates=[0.75],
                offset=Coordinate(z=-0.5))
  # lh.mix([cell], volume=75, repetitions=3, flow_rate=3,
   #         offset=Coordinate(z=0.5))
    lh.discard_tips()

#lh.finish()  # wrap‑up (home, log, etc.)

/var/folders/qd/wlv112pj7rj3g8fv9q2tcw280000gn/T/ipykernel_22741/2012153131.py:3: RuntimeWarning: coroutine 'LiquidHandler.pick_up_tips' was never awaited
  lh.pick_up_tips(next(tips))
/var/folders/qd/wlv112pj7rj3g8fv9q2tcw280000gn/T/ipykernel_22741/2012153131.py:4: RuntimeWarning: coroutine 'LiquidHandler.aspirate' was never awaited
  lh.aspirate([luciferase], [LUC_VOL], flow_rates=[0.75])
/var/folders/qd/wlv112pj7rj3g8fv9q2tcw280000gn/T/ipykernel_22741/2012153131.py:6: RuntimeWarning: coroutine 'LiquidHandler.dispense' was never awaited
  lh.dispense([cell], [LUC_VOL], flow_rates=[0.75],
/var/folders/qd/wlv112pj7rj3g8fv9q2tcw280000gn/T/ipykernel_22741/2012153131.py:10: RuntimeWarning: coroutine 'LiquidHandler.discard_tips' was never awaited
  lh.discard_tips()


In [10]:
lh.summary()


Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: Empty       | 11: tiprack_11  | 12: trash_co... |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: Empty       |  8: tiprack_8   |  9: waste_res   |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: tiprack_4   |  5: Empty       |  6: working_... |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: tiprack_1   |  2: Empty       |  3: reagent_... |
|                 |                 |                 |
+-----------------+-----------------+-----------------+

